# Automatic Syllabus Classification

In [1]:
from syllabus_classificaiton.functions import load_curriculum
from syllabus_classificaiton.functions import transform_curriculum
from syllabus_classificaiton.prompt import prompt
from ollama import chat
from os.path import join

In [2]:
file = "01_machine_learning.json"

path = join("data", file)

curriculum = transform_curriculum(
    load_curriculum(path)
)

response = chat(
    model='digicomp',
    messages=[{
      'role': 'user',
      'content': prompt + curriculum}],
    stream=False,
)

with open(join("ergebnisse", file.split(".")[0]+".md"), "w") as output_file:
  output_file.write(response.message.content)